<a href="https://colab.research.google.com/github/abiaggi/artificial-flavor-gpt2/blob/main/KappoloYT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
CARD_FORMAT = "Name: {0} | Rarity: {1} | Cost: {2} | Type: {3} | Stats: {4} - {5} | Text: {6} | FlavorText: {7} | Loyalty: {8}"
def card2out(card):
  return CARD_FORMAT.format(card.get('name'), card.get('rarity'), card.get('manaCost'), card.get('type'), card.get('power'), card.get('toughness'), card.get('text'), card.get('flavorText'), card.get('loyalty'))

In [3]:
import os
import sys
import requests
import json
from tqdm import tqdm

path = "json"
file_name = "cards.json"

if not os.path.exists(path):
    os.makedirs(path)

url = "https://mtgjson.com/api/v5/AllPrintings.json"
r = requests.get(url, stream=True)
with open(os.path.join(path, file_name), 'wb') as f:
    f.write(r.content)

In [4]:
import json
with open(os.path.join(path, file_name), encoding='utf-8') as f:
    all_cards = json.load(f)

card_dict = {}

for set_exp in all_cards['data']:
    set_cards = all_cards['data'][set_exp]['cards'];
    for card in set_cards:
       card_dict[card['name']] = card2out(card)

if not os.path.exists('out'):
    os.makedirs('out')
with open(os.path.join('out', 'cards.txt'), 'w', encoding='utf-8') as f:
    for card_name in card_dict:
        f.write(' '.join(card_dict[card_name].split()) + '\n')

In [5]:
model = "117M"

subdir = os.path.join('models', model)
if not os.path.exists(subdir):
    os.makedirs(subdir)
subdir = subdir.replace('\\','/') # needed for Windows

for filename in ['checkpoint','encoder.json','hparams.json','model.ckpt.data-00000-of-00001', 'model.ckpt.index', 'model.ckpt.meta', 'vocab.bpe']:

    r = requests.get("https://openaipublic.blob.core.windows.net/gpt-2/" + subdir + "/" + filename, stream=True)

    with open(os.path.join(subdir, filename), 'wb') as f:
        file_size = int(r.headers["content-length"])
        chunk_size = 1000
        with tqdm(ncols=100, desc="Fetching " + filename, total=file_size, unit_scale=True) as pbar:
            # 1k for chunk_size, since Ethernet packet size is around 1500 bytes
            for chunk in r.iter_content(chunk_size=chunk_size):
                f.write(chunk)
                pbar.update(chunk_size)

Fetching checkpoint: 1.00kit [00:00, 277kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 4.79Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 562kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:09, 50.1Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 1.55Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 3.68Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 2.72Mit/s]                                                       


In [6]:
pip install tensorflow==1.15

In [7]:
pip install gpt2-client

In [8]:
from gpt2_client import GPT2Client

gpt2 = GPT2Client('117M')  # This could also be `345M`, `774M`, or `1558M`.
gpt2.load_model(force_download=False)  # Use cached versions if available.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Loading checkpoint                             File already exists 
Loading encoder.json                           File already exists 
Loading hparams.json                           File already exists 
Loading model.ckpt.data-00000-of-00001         File already exists 
Loading model.ckpt.index                       File already exists 
Loading model.ckpt.meta                        File already exists 
Loading vocab.bpe                              File already exists 


In [9]:
corpus = "out/cards.txt"
custom = gpt2.finetune(corpus, return_text=True)

Loading checkpoint models/117M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:09<00:00,  9.39s/it]


dataset has 1984946 tokens
Training...
[1 | 9.20] loss=1.70 avg=1.70
[2 | 12.27] loss=1.71 avg=1.70
[3 | 15.32] loss=2.09 avg=1.83
[4 | 18.38] loss=1.79 avg=1.82
[5 | 21.45] loss=1.77 avg=1.81
[6 | 24.52] loss=1.58 avg=1.77
[7 | 27.57] loss=1.56 avg=1.74
[8 | 30.64] loss=1.65 avg=1.73
[9 | 33.71] loss=1.65 avg=1.72
[10 | 36.78] loss=1.61 avg=1.71
[11 | 39.86] loss=1.39 avg=1.68
[12 | 42.92] loss=1.50 avg=1.66
[13 | 46.00] loss=1.71 avg=1.67
[14 | 49.05] loss=1.49 avg=1.65
[15 | 52.13] loss=1.53 avg=1.64
[16 | 55.20] loss=1.51 avg=1.63
[17 | 58.29] loss=1.47 avg=1.62
[18 | 61.38] loss=1.41 avg=1.61
[19 | 64.45] loss=1.46 avg=1.60
[20 | 67.56] loss=1.47 avg=1.59
[21 | 70.65] loss=1.53 avg=1.59
[22 | 73.73] loss=1.57 avg=1.59
[23 | 76.81] loss=1.42 avg=1.58
[24 | 79.90] loss=1.80 avg=1.59
[25 | 82.97] loss=1.54 avg=1.59
[26 | 86.03] loss=1.47 avg=1.58
[27 | 89.13] loss=1.31 avg=1.57
[28 | 92.20] loss=1.64 avg=1.58
[29 | 95.28] loss=1.32 avg=1.57
[30 | 98.37] loss=1.55 avg=1.57
[31 | 101.4